# Exploring Role Skill Expression in League

After reading the [Oracle's Elixir blog post](https://oracleselixir.com/2020/02/luck-and-league-measuring-ideal-season-length-in-lol/) I wanted to apply the same approach to roles in League of Legends. I recalculate the results of the blog post, update the half skill half luck breakpoint with the addition of 2020 data, and then apply the approach to roles in a game.

In [282]:
import math
import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 500

In [283]:
games_2018 = pd.read_csv("../data/2018_LoL_esports_match_data_from_OraclesElixir_20201019.csv")
games_2019 = pd.read_csv("../data/2019_LoL_esports_match_data_from_OraclesElixir_20201019.csv")
games_2020 = pd.read_csv("../data/2020_LoL_esports_match_data_from_OraclesElixir_20201019.csv")

In [284]:
games_2018 = games_2018.replace('NA LCS', 'LCS')

### Validate Function

Recalculating the results from the work of the original [Oracle's Elixir blog post](https://oracleselixir.com/2020/02/luck-and-league-measuring-ideal-season-length-in-lol/) to ensure that I have my math down right.

In [285]:
df = games_2018.append(games_2019, ignore_index=True)

In [286]:
df.shape

(148442, 105)

In [287]:
df = df[df['league'].isin(['LCS', 'LEC'])]
df['datacompleteness'].value_counts()

complete    8292
partial       24
Name: datacompleteness, dtype: int64

In [288]:
df = df[df['datacompleteness'] == 'complete']
df = df[df['playoffs'] == 0]
df.shape

(6624, 105)

In [289]:
df.head(10)

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
1692,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,1,Blue,top,Impact,Team Liquid,Gangplank,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,3,0,6,11,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,16858,599.5732,0.367870,386.8880,319.9881,14,0.4979,2,0.0711,2,24.0,0.8536,13474,9757,347.0184,0.239742,12133.0,NaN,254.0,248.0,6,1,4,9.0338,3650.0,4637.0,77.0,3119.0,4254.0,78.0,531.0,383.0,-1.0,5824.0,7885.0,130.0,5087.0,7378.0,140.0,737.0,507.0,-10.0
1693,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,2,Blue,jng,Xmithie,Team Liquid,Gragas,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,0,0,9,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4355,154.8903,0.095033,778.6841,808.5951,22,0.7825,9,0.3201,9,56.0,1.9917,9998,6281,223.3906,0.154331,8325.0,NaN,162.0,26.0,136,97,16,5.7617,2801.0,3194.0,50.0,3190.0,3707.0,70.0,-389.0,-513.0,-20.0,4641.0,5945.0,88.0,4643.0,5427.0,101.0,-2.0,518.0,-13.0
1694,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,3,Blue,mid,Pobelter,Team Liquid,Malzahar,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,2,1,7,11,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,10390,369.5317,0.226727,251.2033,219.6562,17,0.6046,5,0.1778,6,34.0,1.2092,12316,8599,305.8328,0.211288,10950.0,NaN,278.0,260.0,18,13,4,9.8874,3818.0,4866.0,95.0,3620.0,5005.0,103.0,198.0,-139.0,-8.0,5897.0,7708.0,146.0,5540.0,8006.0,162.0,357.0,-298.0,-16.0
1695,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,4,Blue,bot,Doublelift,Team Liquid,Tristana,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,5,0,5,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11534,410.2193,0.251691,168.4410,361.2448,14,0.4979,11,0.3912,3,40.0,1.4226,14046,10329,367.3622,0.253797,12375.0,NaN,289.0,253.0,36,13,8,10.2786,3510.0,3360.0,94.0,3563.0,3173.0,97.0,-53.0,187.0,-3.0,6213.0,5507.0,142.0,5395.0,5280.0,155.0,818.0,227.0,-13.0
1696,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,5,Blue,sup,Olleh,Team Liquid,Taric,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,1,0,9,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,2689,95.6372,0.058678,229.5791,516.8109,47,1.6716,5,0.1778,16,70.0,2.4896,9449,5732,

In [290]:
validation_df = df.groupby(['team', 'year', 'split']).mean().reset_index()

In [291]:
validation_df.shape

(60, 91)

In [292]:
validation_df.head()

,team,year,split,playoffs,game,patch,playerid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
0,100 Thieves,2018,Spring,0.0,1.0,8.028947,25.745614,2346.315789,0.684211,3.368421,2.719298,9.122807,10.105263,8.157895,0.315789,0.035088,0.000000,0.000000,0.429825,0.136842,0.242105,0.063158,0.278032,0.488411,0.684211,2.263158,1.789474,2.105263,1.368421,0.368421,0.473684,0.421053,0.842105,NaN,0.157895,0.421053,0.473684,0.473684,0.421053,0.578947,1.263158,0.947368,0.578947,7.578947,5.578947,0.578947,0.631579,0.526316,0.263158,24681.403509,615.379377,0.2,917.382196,1031.726074,53.596491,1.352514,24.315789,0.601667,14.578947,112.631579,2.802554,23485.736842,15048.947368,386.413446,0.2,21420.684211,-0.001495,270.852632,382.157895,69.263158,44.754386,6.824561,11.550933,5242.263158,6404.508772,112.017544,5233.859649,6346.000000,113.035088,8.403509,58.508772,-1.017544,8094.192982,10121.333333,178.543860,8066.140351,10077.298246,180.719298,28.052632,44.035088,-2.175439
1,100 Thieves,2018,Summer,0.0,1.0,8.134000,28.541667,2227.850000,0.600000,4.583333,3.633333,10.833333,13.750000,10.900000,0.400000,0.083333,0.016667,0.016667,0.308333,0.120000,0.130000,0.080000,0.370485,0.677330,0.500000,1.800000,2.050000,1.550000,1.850000,0.250000,0.550000,0.200000,0.550000,NaN,0.250000,0.200000,0.350000,0.350000,0.600000,0.500000,0.950000,0.550000,0.500000,7.450000,5.550000,0.500000,0.400000,0.500000,0.250000,23914.383333,622.876907,0.2,921.214407,785.926151,39.433333,1.044353,18.833333,0.493352,15.350000,97.950000,2.590339,22614.333333,14580.308333,391.557609,0.2,20825.383333,0.047090,247.280000,340.633333,71.500000,46.750000,8.616667,11.097427,5109.483333,6298.433333,104.283333,5170.066667,6274.050000,106.833333,-60.583333,24.383333,-2.550000,8080.266667,10215.333333,167.500000,8038.666667,10059.350000,170.116667,41.600000,155.983333,-2.616667
2,100 Thieves,2019,Spring,0.0,1.0,9.032222,28.425926,2153.333333,0.222222,2.537037,3.703704,5.648148,7.611111,11.111111,0.259259,0.055556,0.000000,0.000000,0.194444,0.066667,0.100000,0.133333,0.206589,0.540056,0.388889,2.000000,2.888889,1.777778,2.611111,0.277778,0.722222,0.277778,0.500000,NaN,0.222222,0.277778,0.388889,0.388889,0.611111,0.333333,0.611111,1.000000,0.444444,4.444444,8.055556,0.333333,0.333333,0.259259,0.546296,18405.351852,513.482944,0.2,878.342187,901.373933,40.129630,1.108027,17.925926,0.479167,14.611111,89.166667,2.427142,20448.203704,12667.601852,347.064573,0.2,19314.851852,-0.043822,237.233333,326.444444,68.944444,46.814815,6.277778,10.957660,5104.314815,6255.462963,108.074074,5165.629630,6255.185185,108.759259,-61.314815,0.277778,-0.685185,8083.296296,9958.037037,169.481481,8269.296296,10087.944444,172.111111,-186.000000,-129.907407,-2.629630
3,100 Thieves,2019,Summer,0.0,1.0,9.123158,30.131579,2056.157895,0.421053,3.210526,3.842105,8.245614,9.631579,11.526316,0.263158,0.000000,0.000000,0.000000,0.254386,0.084211,0.136842,0.115789,0.277289,0.613232,0.315789,1.578947,2.736842,1.473684,2.684211,0.157895,0.526316,0.473684,0.315789,NaN,0.105263,0.052632,0.473684,0.473684,0.473684,0.421053,0.526316,0.947368,0.

In [293]:
win_percentage_standard_deviation = validation_df['result'].std()
print(win_percentage_standard_deviation)

0.17078795942724667


In [294]:
games_in_split = 18
random_standard_deviation = math.sqrt(.25 / games_in_split)
print(random_standard_deviation)

0.11785113019775792


In [295]:
#var_observed = var_true + var_random
var_observed = win_percentage_standard_deviation ** 2
var_random = random_standard_deviation ** 2
var_true = var_observed - var_random
print("var_observed: ", var_observed)
print("var_random: ", var_random)
print("var_true: ", var_true)

var_observed:  0.029168527085322856
var_random:  0.013888888888888888
var_true:  0.015279638196433968


In [296]:
games_breakpoint = .25 / (var_true)
print(games_breakpoint)

16.361643959498082


In [297]:
def get_games_breakpoint(win_pct_column, num_games_season):
    win_percentage_standard_deviation = win_pct_column.std()
    random_standard_deviation = math.sqrt(.25 / num_games_season)
    var_observed = win_percentage_standard_deviation ** 2
    var_random = random_standard_deviation ** 2
    var_true = var_observed - var_random
    games_breakpoint = .25 / (var_true)
    return games_breakpoint

In [298]:
breakpoint = get_games_breakpoint(validation_df['result'], 18)
print(breakpoint)

16.361643959498082


### Run Function with 2020 Data

In [299]:
games = games_2018.append(games_2019, ignore_index=True)
games = games.append(games_2020, ignore_index=True)
games.shape

(218294, 105)

In [300]:
df = games[games['league'].isin(['LCS', 'LEC'])]
df['datacompleteness'].value_counts()

complete    14280
partial        96
Name: datacompleteness, dtype: int64

In [301]:
df = df[df['datacompleteness'] == 'complete']
df = df[df['playoffs'] == 0]
df.shape

(10992, 105)

In [302]:
df.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,playerid,side,position,player,team,champion,ban1,ban2,ban3,ban4,ban5,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
1692,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,1,Blue,top,Impact,Team Liquid,Gangplank,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,3,0,6,11,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,16858,599.5732,0.367870,386.8880,319.9881,14,0.4979,2,0.0711,2,24.0,0.8536,13474,9757,347.0184,0.239742,12133.0,NaN,254.0,248.0,6,1,4,9.0338,3650.0,4637.0,77.0,3119.0,4254.0,78.0,531.0,383.0,-1.0,5824.0,7885.0,130.0,5087.0,7378.0,140.0,737.0,507.0,-10.0
1693,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,2,Blue,jng,Xmithie,Team Liquid,Gragas,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,0,0,9,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,4355,154.8903,0.095033,778.6841,808.5951,22,0.7825,9,0.3201,9,56.0,1.9917,9998,6281,223.3906,0.154331,8325.0,NaN,162.0,26.0,136,97,16,5.7617,2801.0,3194.0,50.0,3190.0,3707.0,70.0,-389.0,-513.0,-20.0,4641.0,5945.0,88.0,4643.0,5427.0,101.0,-2.0,518.0,-13.0
1694,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,3,Blue,mid,Pobelter,Team Liquid,Malzahar,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,2,1,7,11,1,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,10390,369.5317,0.226727,251.2033,219.6562,17,0.6046,5,0.1778,6,34.0,1.2092,12316,8599,305.8328,0.211288,10950.0,NaN,278.0,260.0,18,13,4,9.8874,3818.0,4866.0,95.0,3620.0,5005.0,103.0,198.0,-139.0,-8.0,5897.0,7708.0,146.0,5540.0,8006.0,162.0,357.0,-298.0,-16.0
1695,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,4,Blue,bot,Doublelift,Team Liquid,Tristana,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,5,0,5,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,11534,410.2193,0.251691,168.4410,361.2448,14,0.4979,11,0.3912,3,40.0,1.4226,14046,10329,367.3622,0.253797,12375.0,NaN,289.0,253.0,36,13,8,10.2786,3510.0,3360.0,94.0,3563.0,3173.0,97.0,-53.0,187.0,-3.0,6213.0,5507.0,142.0,5395.0,5280.0,155.0,818.0,227.0,-13.0
1696,TRLH1/1002440062,complete,http://matchhistory.na.leagueoflegends.com/en/...,LCS,2018,Spring,0,2018-01-20 20:50:09,1,8.01,5,Blue,sup,Olleh,Team Liquid,Taric,Ornn,Zoe,Azir,Thresh,Kha'Zix,1687,1,1,0,9,11,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3912,0.4268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,2689,95.6372,0.058678,229.5791,516.8109,47,1.6716,5,0.1778,16,70.0,2.4896,9449,5732,

In [303]:
season_length_breakpoint_df = df.groupby(['team', 'year', 'split']).mean().reset_index()
season_length_breakpoint_df.head(10)

,team,year,split,playoffs,game,patch,playerid,gamelength,result,kills,deaths,assists,teamkills,teamdeaths,doublekills,triplekills,quadrakills,pentakills,firstblood,firstbloodkill,firstbloodassist,firstbloodvictim,team kpm,ckpm,firstdragon,dragons,opp_dragons,elementaldrakes,opp_elementaldrakes,infernals,mountains,clouds,oceans,dragons (type unknown),elders,opp_elders,firstherald,heralds,opp_heralds,firstbaron,barons,opp_barons,firsttower,towers,opp_towers,firstmidtower,firsttothreetowers,inhibitors,opp_inhibitors,damagetochampions,dpm,damageshare,damagetakenperminute,damagemitigatedperminute,wardsplaced,wpm,wardskilled,wcpm,controlwardsbought,visionscore,vspm,totalgold,earnedgold,earned gpm,earnedgoldshare,goldspent,gspd,total cs,minionkills,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm,goldat10,xpat10,csat10,opp_goldat10,opp_xpat10,opp_csat10,golddiffat10,xpdiffat10,csdiffat10,goldat15,xpat15,csat15,opp_goldat15,opp_xpat15,opp_csat15,golddiffat15,xpdiffat15,csdiffat15
0,100 Thieves,2018,Spring,0.0,1.0,8.028947,25.745614,2346.315789,0.684211,3.368421,2.719298,9.122807,10.105263,8.157895,0.315789,0.035088,0.000000,0.000000,0.429825,0.136842,0.242105,0.063158,0.278032,0.488411,0.684211,2.263158,1.789474,2.105263,1.368421,0.368421,0.473684,0.421053,0.842105,NaN,0.157895,0.421053,0.473684,0.473684,0.421053,0.578947,1.263158,0.947368,0.578947,7.578947,5.578947,0.578947,0.631579,0.526316,0.263158,24681.403509,615.379377,0.2,917.382196,1031.726074,53.596491,1.352514,24.315789,0.601667,14.578947,112.631579,2.802554,23485.736842,15048.947368,386.413446,0.2,21420.684211,-0.001495,270.852632,382.157895,69.263158,44.754386,6.824561,11.550933,5242.263158,6404.508772,112.017544,5233.859649,6346.000000,113.035088,8.403509,58.508772,-1.017544,8094.192982,10121.333333,178.543860,8066.140351,10077.298246,180.719298,28.052632,44.035088,-2.175439
1,100 Thieves,2018,Summer,0.0,1.0,8.134000,28.541667,2227.850000,0.600000,4.583333,3.633333,10.833333,13.750000,10.900000,0.400000,0.083333,0.016667,0.016667,0.308333,0.120000,0.130000,0.080000,0.370485,0.677330,0.500000,1.800000,2.050000,1.550000,1.850000,0.250000,0.550000,0.200000,0.550000,NaN,0.250000,0.200000,0.350000,0.350000,0.600000,0.500000,0.950000,0.550000,0.500000,7.450000,5.550000,0.500000,0.400000,0.500000,0.250000,23914.383333,622.876907,0.2,921.214407,785.926151,39.433333,1.044353,18.833333,0.493352,15.350000,97.950000,2.590339,22614.333333,14580.308333,391.557609,0.2,20825.383333,0.047090,247.280000,340.633333,71.500000,46.750000,8.616667,11.097427,5109.483333,6298.433333,104.283333,5170.066667,6274.050000,106.833333,-60.583333,24.383333,-2.550000,8080.266667,10215.333333,167.500000,8038.666667,10059.350000,170.116667,41.600000,155.983333,-2.616667
2,100 Thieves,2019,Spring,0.0,1.0,9.032222,28.425926,2153.333333,0.222222,2.537037,3.703704,5.648148,7.611111,11.111111,0.259259,0.055556,0.000000,0.000000,0.194444,0.066667,0.100000,0.133333,0.206589,0.540056,0.388889,2.000000,2.888889,1.777778,2.611111,0.277778,0.722222,0.277778,0.500000,NaN,0.222222,0.277778,0.388889,0.388889,0.611111,0.333333,0.611111,1.000000,0.444444,4.444444,8.055556,0.333333,0.333333,0.259259,0.546296,18405.351852,513.482944,0.2,878.342187,901.373933,40.129630,1.108027,17.925926,0.479167,14.611111,89.166667,2.427142,20448.203704,12667.601852,347.064573,0.2,19314.851852,-0.043822,237.233333,326.444444,68.944444,46.814815,6.277778,10.957660,5104.314815,6255.462963,108.074074,5165.629630,6255.185185,108.759259,-61.314815,0.277778,-0.685185,8083.296296,9958.037037,169.481481,8269.296296,10087.944444,172.111111,-186.000000,-129.907407,-2.629630
3,100 Thieves,2019,Summer,0.0,1.0,9.123158,30.131579,2056.157895,0.421053,3.210526,3.842105,8.245614,9.631579,11.526316,0.263158,0.000000,0.000000,0.000000,0.254386,0.084211,0.136842,0.115789,0.277289,0.613232,0.315789,1.578947,2.736842,1.473684,2.684211,0.157895,0.526316,0.473684,0.315789,NaN,0.105263,0.052632,0.473684,0.473684,0.473684,0.421053,0.526316,0.947368,0.

In [304]:
breakpoint = get_games_breakpoint(season_length_breakpoint_df['result'], 18)
print(breakpoint)

14.365126087736414


### Analyze Roles